# Day 52

Today I'm going to use SQL to rank the top 10 players by total PPR fantasy points through Week 14 of the NFL season and compare against their 2021 rank through the same number of weeks

In [1]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [9]:
query = """
WITH last_year_rankings AS (
    SELECT
        player_display_name,
        position,
        r_pts_ppr_2021
    FROM (
    SELECT
        *,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr DESC) AS r_pts_ppr_2021,
        DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr DESC) AS r_dense_pts_ppr_2021
    -- Grouped and filtered table
    FROM (
        SELECT
            season,
            week,
            player_display_name,
            position,
            ROUND(SUM(fantasy_points),2) AS tot_pts,
            ROUND(SUM(fantasy_points_ppr),2) AS tot_pts_ppr
        FROM weekly
        WHERE position IN ('QB', 'RB', 'WR', 'TE')
            AND season = 2021
            AND week <= 14
        GROUP BY position, player_display_name
        ) AS temp
    ) AS rankings
)
SELECT
    rankings.player_display_name,
    rankings.position,
    tot_pts_ppr,
    r_pts_ppr,
    r_pts_ppr_2021
FROM (
SELECT
    *,
    RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr DESC) AS r_pts_ppr,
    DENSE_RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr DESC) AS r_dense_pts_ppr
-- Grouped and filtered table
FROM (
    SELECT
        season,
        week,
        player_display_name,
        position,
        ROUND(SUM(fantasy_points),2) AS tot_pts,
        ROUND(SUM(fantasy_points_ppr),2) AS tot_pts_ppr
    FROM weekly
    WHERE position IN ('QB', 'RB', 'WR', 'TE')
        AND season = 2022
        AND week <= 14
    GROUP BY position, player_display_name
    ) AS temp
) AS rankings
LEFT JOIN last_year_rankings
    ON last_year_rankings.player_display_name = rankings.player_display_name
    AND last_year_rankings.position = rankings.position
WHERE r_pts_ppr <= 10
"""

df = pd.read_sql(query, conn)

In [10]:
df.query("position == 'QB'")

,player_display_name,position,tot_pts_ppr,r_pts_ppr,r_pts_ppr_2021
0,Jalen Hurts,QB,334.88,1,7
1,Patrick Mahomes,QB,320.40,2,5
2,Josh Allen,QB,314.92,3,1
3,Joe Burrow,QB,290.80,4,10
4,Justin Fields,QB,246.34,5,30
5,Geno Smith,QB,245.62,6,39
6,Lamar Jackson,QB,236.08,7,9
7,Trevor Lawrence,QB,235.88,8,26
8,Justin Herbert,QB,231.54,9,3
9,Daniel Jones,QB,222.16,10,23


In [11]:
df.query("position == 'RB'")

,player_display_name,position,tot_pts_ppr,r_pts_ppr,r_pts_ppr_2021
10,Austin Ekeler,RB,292.70,1,2
11,Josh Jacobs,RB,282.80,2,16
12,Christian McCaffrey,RB,268.56,3,30
13,Derrick Henry,RB,245.66,4,9
14,Saquon Barkley,RB,226.40,5,32
15,Nick Chubb,RB,224.50,6,19
16,Tony Pollard,RB,218.30,7,26
17,Aaron Jones,RB,201.50,8,11
18,Rhamondre Stevenson,RB,200.70,9,51
19,Miles Sanders,RB,199.30,10,46


In [12]:
df.query("position == 'WR'")

,player_display_name,position,tot_pts_ppr,r_pts_ppr,r_pts_ppr_2021
30,Justin Jefferson,WR,296.76,1,2
31,Tyreek Hill,WR,285.00,2,4
32,Davante Adams,WR,278.60,3,3
33,Stefon Diggs,WR,277.90,4,10
34,A.J. Brown,WR,223.00,5,41
35,Amon-Ra St. Brown,WR,216.70,6,45
36,CeeDee Lamb,WR,210.70,7,14
37,Tyler Lockett,WR,206.60,8,16
38,Cooper Kupp,WR,201.40,9,1
39,Christian Kirk,WR,197.10,10,27


In [13]:
df.query("position == 'TE'")

,player_display_name,position,tot_pts_ppr,r_pts_ppr,r_pts_ppr_2021
20,Travis Kelce,TE,257.4,1,2
21,Mark Andrews,TE,157.9,2,1
22,T.J. Hockenson,TE,155.7,3,4
23,Evan Engram,TE,133.8,4,24
24,Pat Freiermuth,TE,128.0,5,12
25,Zach Ertz,TE,115.6,6,10
26,David Njoku,TE,115.5,7,20
27,George Kittle,TE,114.0,8,3
28,Dallas Goedert,TE,113.4,9,11
29,Gerald Everett,TE,111.0,10,23
